<a href="https://colab.research.google.com/github/melsubair/Subair/blob/master/covid_19_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import scipy
import warnings
import numpy as np
from tqdm import tqdm
from random import shuffle
import matplotlib.pyplot as plt

from keras.optimizers import Adam, RMSprop
from keras.models import Sequential, Model
from keras.layers import  Conv2D, MaxPooling2D, Activation, Flatten,Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

Using TensorFlow backend.


In [2]:
# Filter Warnings
warnings.filterwarnings("ignore")

In [3]:
# File Paths
input_path = "../covid-19-xray/"

In [4]:
# File Contents
for _set in ['train', 'validation']:
    normal = len(os.listdir(input_path + _set + '/normal'))
    pneumonia = len(os.listdir(input_path + _set + '/pneumonia'))
    print('The {} folder contains {} Normal and {} Pneumonia images.'.format(_set, normal, pneumonia))

The train folder contains 1336 Normal and 1615 Pneumonia images.
The validation folder contains 8 Normal and 8 Pneumonia images.
